In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to 

In [2]:
# model = Qwen2VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2-VL-72B-Instruct", torch_dtype="auto", device_map="auto").eval()


model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-72B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
).eval()

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 38/38 [08:01<00:00, 12.67s/it]


In [26]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-72B-Instruct")

In [ ]:
path='/root/home/data/pos_collage_903.jpg'
messages = [
    {"role": "user","content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},],}]


# Preparation for inference
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor( text=[text], images=image_inputs, padding=True, return_tensors="pt",)


In [43]:
inputs = processor( text=[text], images=image_inputs, padding=True, return_tensors="pt",)
inputs = inputs.to("cuda")

In [44]:
generated_ids = model.generate(**inputs, max_new_tokens=128)

In [45]:
generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]

In [46]:
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(output_text)

['The image depicts a serene beach scene at sunset. A person is sitting on the sand, facing the ocean, with their back to the camera. They are wearing a plaid shirt and shorts. Next to them, a large dog, possibly a Labrador Retriever, is sitting upright, facing the person. The dog is wearing a harness. The sun is setting in the background, casting a warm glow over the entire scene, creating a peaceful and tranquil atmosphere. The waves gently lap at the shore, adding to the calm ambiance.']


In [ ]:
["The image shows a close-up of a person's hand holding a smartphone. The screen of the smartphone displays a map application with various locations marked and routes indicated. The map appears to be centered around a specific area, possibly for navigation or location tracking purposes. The background is blurred, focusing attention on the smartphone screen."]

In [17]:
output_text==["The image shows a close-up of a person's hand holding a smartphone. The screen of the smartphone displays a map application with various locations marked and routes indicated. The map appears to be centered around a specific area, possibly for navigation or location tracking purposes. The background is blurred, focusing attention on the smartphone screen."]

True